<a href="https://colab.research.google.com/github/estebanhernandezr/DNA-compression/blob/master/gzip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PDF2TXT**

In [1]:
pip install slate3k

     |████████████████████████████████| 100 kB 7.8 MB/s 
     |████████████████████████████████| 49 kB 4.6 MB/s 


In [2]:
pip install PyPDF3

     |████████████████████████████████| 294 kB 22.1 MB/s 
  Created wheel for PyPDF3: filename=PyPDF3-1.0.6-py3-none-any.whl size=62885 sha256=a5198d422ddd53ec163422e5dfc183619327e81f23f2f3c03bcda230bb656d60
  Stored in directory: /root/.cache/pip/wheels/98/e9/28/80877f24d385035bf56cb22f12d7653203032491326858f1ce
Successfully built PyPDF3


In [3]:
import PyPDF3
import slate3k as slate

In [4]:
def pdf2txt(filename, text='', ini=0, fin=0):
    pdffileobj=open(filename,'rb')
    pdfreader=PyPDF3.PdfFileReader(pdffileobj)
    y=pdfreader.numPages
    #print("num_pages:", y)
    if ini < 0:
        x = y+ini
    else:
        x = ini
    if fin < 0:
        y = y+fin
    else:
        y = y-fin
    print("X:", x, "--->", "Y:", y)
    for i in range(x, y):
        pageobj=pdfreader.getPage(i)
        print("appending...", pageobj.extractText())
        text+=pageobj.extractText()
    #print("TEXTO:", text)
    return text

# **GZIP**

In [5]:
pip install bitarray

     |████████████████████████████████| 88 kB 4.9 MB/s 
  Created wheel for bitarray: filename=bitarray-2.3.6-cp37-cp37m-linux_x86_64.whl size=172603 sha256=be3e1ebb5bd000e2524392814da873e4561b5b6be0949c931416f41203b46d9c
  Stored in directory: /root/.cache/pip/wheels/91/71/89/bcc0df88de1a84543430ca9d4a43ede8c6bdef0f173e2c7ba9
Successfully built bitarray


In [6]:
from bitarray import *
from typing import BinaryIO, Dict, Sequence, Tuple

In [7]:
# FUNCTIONS

def data_from_file(filename: str) -> BinaryIO:
    with open(filename, 'rb') as input_file:
        filedata = input_file.read()
    return filedata

def file_from_bin(filename: str, buffer: bytearray) -> None:
    with open(filename, 'wb') as outfile:
        outfile.write(buffer)
        return None

def bin_from_file(filename: str) -> BinaryIO:
    filedata: bytearray = bitarray(endian='big')
    with open(filename, 'rb') as input_file:
        filedata.fromfile(input_file)
    return filedata

def inipad(symb: int, padsize: int, cad: str) -> BinaryIO:
    pad: bytearray = bytearray(chr(symb), 'utf-8')
    for i in range(padsize-1):
        pad.append(pad[0])
    return pad + cad

In [8]:
# COMPRESSER CLASS

class Compresser:
    _n: int
    _Ls: int
    _symb: int
    _compressed_string: bytearray

    _huffman_dictionary: Dict
    _compressed_huffman: bytearray

    def __init__(self, n: int, Ls: int):
        if (n < 0 or Ls < 0):
			      raise ValueError("Negative buffer sizes")

        self._n = n
        self._Ls = Ls
        self._symb = 32
        self._compressed_string = bitarray(endian='big')

        self._huffman_dictionary = {}
        self._compressed_huffman = bitarray(endian='big')

    def codify_word(self, pos: int, size: int, char: chr=None) -> str:
        n: int = self._n
        Ls: int = self._Ls
        codeword = "{0:0{width}b}".format(pos, width=len("{0:b}".format(n)))
        codeword += "{0:0{width}b}".format(size, width=len("{0:b}".format(Ls)))
        return codeword

    def rep_extension(self, search: bytearray, lookahead: bytearray) -> Sequence[int]:
        n: int = self._n
        Ls: int = self._Ls
        pos: int = -1
        size: int = 0
        char: chr = ''
        for prefixsize in range(1, min(n-Ls, len(lookahead))):
            prefix: str = lookahead[:prefixsize]
            p: int = search.rfind(prefix, 0, (n-Ls)+prefixsize-1)
            if p >= 0:
                pos = p
                size = prefixsize
                char = lookahead[size]
            else:
                break
        return pos, size, char

    def codify_cad(self, cad: str) -> None:
        n: int = self._n
        Ls: int = self._Ls
        symb: int = self._symb
        dictionary: Dict = self._huffman_dictionary

        pcad: bytearray = inipad(symb, n-Ls, cad)
        i: int = 0
        while i < len(pcad)-(n-Ls):
            triple: Sequence[int] = self.rep_extension(pcad[i:i+n], pcad[i+n-Ls:i+n])
            pos: int = triple[0]
            size: int = triple[1]
            if (pos >= 0 and size > 1):
                self._compressed_string.append(True)
                if "{0:0{width}b}".format(pos, width=len("{0:b}".format(n))) in dictionary:
                    huffman_code: str = dictionary["{0:0{width}b}".format(pos,
                                                                          width=len("{0:b}".format(n)))]
                    bin_code: str = huffman_code + "{0:0{width}b}".format(size,
                                                                          width=len("{0:b}".format(Ls)))
                elif len(dictionary) == 0:
                    bin_code: str = self.codify_word(pos, size)
                
                for bit in bin_code:
                    if bit == '1':
                        self._compressed_string.append(True)
                    else:
                        self._compressed_string.append(False)
                i += size
            else:
                self._compressed_string.append(False)
                self._compressed_string.frombytes(bytes([pcad[i+n-Ls]]))
                i += 1
        return None

    def compress(self, filename: str) -> None:
        filedata: bytearray = data_from_file(filename) #CHECKED
        
        #distances = get_distances(filedata) #CHECKED - HUFFMAN PART
        #counts = bl_count_from_distances(distances) #CHECKED - HUFFMAN PART
        #huffman_tree = create_huffman_tree(counts) #CHECKED - HUFFMAN PART
        #dictionary = dictionary_from_tree(huffman_tree, '', codeDictionary) #CHECKED - HUFFMAN PART
        #self._huffman_dictionary = dictionary

        #self.codify_huffmantree(huffman_tree, treeBuffer) #CHECKED - HUFFMAN PART
        self.codify_cad(filedata) #CHECKED

        return None

In [9]:
# DECOMPRESSER CLASS

class Decompresser:
    _n: int
    _Ls: int
    _symb: int
    _decompressed_string: bitarray
    
    _huffman_dictionary: Dict
    _decompressed_huffman: bitarray

    def __init__(self, n: int, Ls: int):
        if (n < 0 or Ls < 0):
			      raise ValueError("Negative buffer sizes")

        self._n = n
        self._Ls = Ls
        self._symb = 32
        self._decompressed_string = bitarray(endian='big')

        self._huffman_dictionary = {}
        self._decompressed_huffman = bitarray(endian='big')

    def decompress_cad(self, filedata: bytearray) -> None:
        n: int = self._n
        Ls: int = self._Ls
        symb: int = self._symb
        self._decompressed_string = inipad(symb, n-Ls, bytes())

        k: int = 0
        while len(filedata) >= 9:
            flag_pair = filedata.pop(0)
            if not flag_pair:
                byte = filedata[0:8].tobytes()
                self._decompressed_string += byte
                del filedata[0:8]
                k += 1
            else:
                position = ''
                for i in range(0, len("{0:b}".format(n))):
                    bit = filedata.pop(0)
                    if bit == True:
                        position += '1'
                    else:
                        position += '0'

                curbitsubstring = ''
                stop = False
                while len(self._huffman_dictionary) > 0 and stop == False:
                    bit = filedata.pop(0)
                    if bit == True:
                        curbitsubstring+="1"
                    else:
                        curbitsubstring+="0"
                    for key in self._huffman_dictionary:
                        if self._huffman_dictionary[key] == str(curbitsubstring):
                            position = key
                            stop = True

                length = ''
                for i in range(0, len("{0:b}".format(Ls))):
                    bit = filedata.pop(0)
                    if bit == True:
                        length += '1'
                    else:
                        length += '0'

                bestDistance = int(position, 2)
                bestLength = int(length, 2)
                for i in range(bestLength):
                    self._decompressed_string.append(
                        self._decompressed_string[k+bestDistance+i])
                k += bestLength

        self._decompressed_string = self._decompressed_string[n-Ls:]
        return None

    def decompress(self, filename: str):
        filedata: bytearray = bin_from_file(filename)

        #root = decodify_huffman_tree(filedata) - HUFFMAN PART
        #dictionary_from_tree(root, '', codeDictionary) - HUFFMAN PART
        
        self.decompress_cad(filedata)

# **TEST SUITE**

In [10]:
filename1 = 'PDFs/Spanish [Spain].pdf'
filename2 = 'PDFs/Italian [Italy].pdf'
text1 = pdf2txt(filename1)
text2 = pdf2txt(filename2, text1, ini=-2, fin=0)

file=open('datos'+'.txt', 'w')
file.write(text2)

X: 0 ---> Y: 9
appending...  
Declaraci
!
n Universal de Derechos Humanos 
Adoptada y proclamada por la As
amblea General en su resoluci
!
n 217 A (III), de 
10 de diciembre de 1948  
Pre
"
mbulo  
Considerando que la libertad, 
la justicia y la paz en el mundo tienen por base el 
reconocimiento de la dignidad intr
#
nseca y de los derechos iguales e inalienables 
de todos los miembros de la familia humana,  
Considerando que el desconocimiento 
y el menosprecio de los derechos 
humanos han originado actos de barbarie ul
trajantes para la conciencia de la 
humanidad; y que se ha procla
mado, como la aspiraci
!
n m
"
s elevada del 
hombre, el advenimiento 
de un mundo en que los seres humanos, liberados del 
temor y de la miseria, disfruten de la
 libertad de palabra y de la libertad de 
creencias,  
Considerando esencial que los derechos
 humanos sean protegidos por un 
r
$
gimen de Derecho, a fin de que el hom
bre no se vea compelido al supremo 
recurso de la rebeli
!
n contra la tira

15446

In [11]:
filename: str = 'datos.txt'

Compresor = Compresser(5000, 1000)
Compresor.compress(filename)

Compresor._compressed_string.fill()
file_from_bin('compressed_file', Compresor._compressed_string)

In [12]:
filename: str = 'compressed_file'

Decompresor = Decompresser(5000, 1000)
Decompresor.decompress(filename)

file_from_bin('decompressed_file', Decompresor._decompressed_string)

# **MATRIX CONSTRUCTION**

In [13]:
import os

In [14]:
def calculate_delta(txt1, txt2, dicc):
    L1 = dicc[txt1]
    L2 = dicc[txt2]
    delta = L1 - L2
    return delta

def calculate2(Aa, Ab, Ba, Bb, A, B, dicc):
    deltaAb = calculate_delta(Ab, A, dicc)
    deltaAa = calculate_delta(Aa, A, dicc)
    deltaBa = calculate_delta(Ba, B, dicc)
    deltaBb = calculate_delta(Bb, B, dicc)
    
    return ((deltaAb - deltaBb)/deltaBb)+((deltaBa-deltaAa)/deltaAa)


In [15]:
def relat_entropy(pdf_1, pdf_2, n, Ls):
    pdffileobj_1=open(pdf_1,'rb')
    pdfreader_1=PyPDF3.PdfFileReader(pdffileobj_1)
    y_1=pdfreader_1.numPages
    txt_1A = pdf2txt(pdf_1, fin=-2)
    txt_1Aa = pdf2txt(pdf_1, txt_1A, ini=-2, fin=0)

    pdffileobj_2=open(pdf_2,'rb')
    pdfreader_2=PyPDF3.PdfFileReader(pdffileobj_2)
    y_2=pdfreader_2.numPages
    txt_2B = pdf2txt(pdf_2, fin=-2)
    txt_2Bb = pdf2txt(pdf_2, txt_2B, ini=-2, fin=0)

    txt_1Ab = pdf2txt(pdf_2, txt_1A, ini=-2, fin=0)
    txt_2Ba = pdf2txt(pdf_1, txt_2B, ini=-2, fin=0)

    file_A=open("TXTs/A.txt", 'w')
    file_A.write(txt_1A)

    file_Aa=open("TXTs/Aa.txt", 'w')
    file_Aa.write(txt_1Aa)

    file_B=open("TXTs/B.txt", 'w')
    file_B.write(txt_2B)

    file_Bb=open("TXTs/Bb.txt", 'w')
    file_Bb.write(txt_2Bb)
    
    file_Ab=open("TXTs/Ab.txt", 'w')
    file_Ab.write(txt_1Ab)

    file_Ab=open("TXTs/Ba.txt", 'w')
    file_Ab.write(txt_2Ba)

    dictionario = {}

    Compresor_A = Compresser(n, Ls)
    Compresor_A.compress("TXTs/A.txt")
    dictionario["TXTs/A.txt"]=len(Compresor_A._compressed_string)

    Compresor_B = Compresser(n, Ls)
    Compresor_B.compress("TXTs/B.txt")
    dictionario["TXTs/B.txt"]=len(Compresor_B._compressed_string)

    Compresor_Aa = Compresser(n, Ls)
    Compresor_Aa.compress("TXTs/Aa.txt")
    dictionario["TXTs/Aa.txt"]=len(Compresor_Aa._compressed_string)

    Compresor_Bb = Compresser(n, Ls)
    Compresor_Bb.compress("TXTs/Bb.txt")
    dictionario["TXTs/Bb.txt"]=len(Compresor_Bb._compressed_string)

    Compresor_Ab = Compresser(n, Ls)
    Compresor_Ab.compress("TXTs/Ab.txt")
    dictionario["TXTs/Ab.txt"]=len(Compresor_Ab._compressed_string)

    Compresor_Ba = Compresser(n, Ls)
    Compresor_Ba.compress("TXTs/Ba.txt")
    dictionario["TXTs/Ba.txt"]=len(Compresor_Ba._compressed_string)

    res = calculate2( 'TXTs/Aa.txt', 'TXTs/Ab.txt', 'TXTs/Ba.txt', 'TXTs/Bb.txt', 'TXTs/A.txt', 'TXTs/B.txt', dictionario)
    print(pdf_1, "(",(res),")", pdf_2)
    return res

In [16]:
search = 2000
ahead = 1000

path = '/content/PDFs/'
dir_path = os.path.dirname(os.path.realpath(path))

PDFs = []
for root, dirs, files in os.walk(dir_path):
    for file_1 in files:
        for file_2 in files:
          if file_1.endswith('.pdf') and file_2.endswith('.pdf'):
              relat_entropy(path+file_1, path+file_2, search, ahead)

X: 0 ---> Y: 7
appending...  
INSON HUQUQLARI UMUMJAHON DEKLARATCIYASI 
1948 yil, 10 dekabrda 
Birlashgan Millatlar Tashkiloti 
Bosh Assambleyasining 217 
!
(Sh) 
ko'rsatmasi orqali qabul qilingan 
va e'lon qilingan.  
Preambula 
Inson oilasi barcha a'zolariga hos bo'l
gan qadr-qimmat hamda ularning teng va 
ajralmas huquqlarini tan olish erkinlik, adolat va yalpi tinchlikning asosi bo'lishini 
e'tiborga olib,  
Inson huquqlarini mensimas
lik va oeq-osti qilish 
odamzod vijdonini o'rtaetgan 
vahshiylarcha qilmishlarga olib kelganini 
xamda kishilar so'z va maslak erkinligi 
sohibi bo'lib, qo'rquv va muhtojlikdan 
holi bo'lib yashaydigan duneni byned etish 
odamlarning nufuzli itilishi, deb e'lon qilinganligini e'tiborga olib,  
Inson so'nggi chora sifatida zulm va istibdodga qarshi isen qilib bosh ko'tarishga 
majbur bo'lmasligining oldini olish maqs
adida inson huquqlarni qonun izmi bilan 
muhofaza etilishi zarurligini e'tiborga olib,  
Xalqlar o'rtasida do'stona munosabatlarn
i rivoj

ZeroDivisionError: ignored

# **DEBUGGIN**

In [17]:
def relat_entropy1(pdf_1, pdf_2, n, Ls):
    m = 2500
    with open(pdf_1, 'rb') as f:
        txt_A = slate.PDF(f)
    #print(txt_A)

    with open(pdf_2, 'rb') as f:
        txt_B = slate.PDF(f)
    #print(txt_B)

    txt_A = str(txt_A)
    txt_B = str(txt_B)
    txt_Aa = str(txt_A + txt_A[:-m])
    txt_Bb = str(txt_B + txt_B[:-m])
    txt_Ab = str(txt_A + txt_B[:-m])
    txt_Ba = str(txt_B + txt_A[:-m])

    file_A=open("TXTs/A.txt", 'w')
    file_A.write(txt_A)

    file_Aa=open("TXTs/Aa.txt", 'w')
    file_Aa.write(txt_Aa)

    file_B=open("TXTs/B.txt", 'w')
    file_B.write(txt_B)

    file_Bb=open("TXTs/Bb.txt", 'w')
    file_Bb.write(txt_Bb)
    
    file_Ab=open("TXTs/Ab.txt", 'w')
    file_Ab.write(txt_Ab)

    file_Ab=open("TXTs/Ba.txt", 'w')
    file_Ab.write(txt_Ba)

    dictionario = {}

    Compresor_A = Compresser(n, Ls)
    Compresor_A.compress("TXTs/A.txt")
    dictionario["TXTs/A.txt"]=len(Compresor_A._compressed_string)

    Compresor_B = Compresser(n, Ls)
    Compresor_B.compress("TXTs/B.txt")
    dictionario["TXTs/B.txt"]=len(Compresor_B._compressed_string)

    Compresor_Aa = Compresser(n, Ls)
    Compresor_Aa.compress("TXTs/Aa.txt")
    dictionario["TXTs/Aa.txt"]=len(Compresor_Aa._compressed_string)

    Compresor_Bb = Compresser(n, Ls)
    Compresor_Bb.compress("TXTs/Bb.txt")
    dictionario["TXTs/Bb.txt"]=len(Compresor_Bb._compressed_string)

    Compresor_Ab = Compresser(n, Ls)
    Compresor_Ab.compress("TXTs/Ab.txt")
    dictionario["TXTs/Ab.txt"]=len(Compresor_Ab._compressed_string)

    Compresor_Ba = Compresser(n, Ls)
    Compresor_Ba.compress("TXTs/Ba.txt")
    dictionario["TXTs/Ba.txt"]=len(Compresor_Ba._compressed_string)

    res = calculate2( 'TXTs/Aa.txt', 'TXTs/Ab.txt', 'TXTs/Ba.txt', 'TXTs/Bb.txt', 'TXTs/A.txt', 'TXTs/B.txt', dictionario)
    #print(pdf_1, "(",(res),")", pdf_2)
    str_res = pdf_1 + "  (" + str(res) + ")  " + pdf_2
    return str_res

In [20]:
search = 2000
ahead = 1000

resultados = []

path = '/content/PDFs/'
dir_path = os.path.dirname(os.path.realpath(path))

PDFs = []
for root, dirs, files in os.walk(dir_path):
    for file_1 in files:
        for file_2 in files:
          if file_1.endswith('.pdf') and file_2.endswith('.pdf'):
              resultados.append(relat_entropy1(path+file_1, path+file_2, search, ahead))

In [21]:
for resultado in resultados:
    print(resultado)

/content/PDFs/Uzbek [Utzbekistan].pdf  (0.0)  /content/PDFs/Uzbek [Utzbekistan].pdf
/content/PDFs/Uzbek [Utzbekistan].pdf  (0.030503116161148376)  /content/PDFs/Portuguese [Portugal].pdf
/content/PDFs/Uzbek [Utzbekistan].pdf  (0.03183581240712207)  /content/PDFs/French [France].pdf
/content/PDFs/Uzbek [Utzbekistan].pdf  (0.030624463327047387)  /content/PDFs/Italian [Italy].pdf
/content/PDFs/Uzbek [Utzbekistan].pdf  (0.030256444494625796)  /content/PDFs/Estonian [Estonia].pdf
/content/PDFs/Uzbek [Utzbekistan].pdf  (0.030149887878850864)  /content/PDFs/Turkish [Turkey].pdf
/content/PDFs/Uzbek [Utzbekistan].pdf  (0.03307942688364267)  /content/PDFs/Spanish [Spain].pdf
/content/PDFs/Portuguese [Portugal].pdf  (0.030503116161148376)  /content/PDFs/Uzbek [Utzbekistan].pdf
/content/PDFs/Portuguese [Portugal].pdf  (0.0)  /content/PDFs/Portuguese [Portugal].pdf
/content/PDFs/Portuguese [Portugal].pdf  (0.020480019200068)  /content/PDFs/French [France].pdf
/content/PDFs/Portuguese [Portugal].pdf